In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import time
import re

GET LATITUDE AND LONGITUDE

In [16]:
def get_latitude_longitude(driver, shop_name):
    
    search_box_xpath = '/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input'
    search_button_xpath = '/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/div[1]/button/span'
    
    try:
        # Clear the search box
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, search_box_xpath))
        )
        
        
        search_box.clear()
        
        # Enter the shop name in the search box
        search_box.send_keys(shop_name)
        
        # Click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, search_button_xpath))
        )
  
        
        search_button.click()
        
        time.sleep(2)
        
        # Wait for the map to update and extract the coordinates from the URL
        WebDriverWait(driver, 10).until(
            EC.url_contains('/@')
        )
        current_url = driver.current_url
        coords = current_url.split('/@')[1].split(',')[0:2]
        latitude, longitude = coords[0], coords[1]
        return latitude, longitude
    except (NoSuchElementException, TimeoutException):
        return None, None

def scrape_lat_long(csv_file_path):

    df = pd.read_csv(csv_file_path)
    
    # Lists to store latitude and longitude
    latitudes = []
    longitudes = []
    
    service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")

    driver = webdriver.Chrome(service=service)

    driver.get("https://www.google.com/maps")
    
    for index, row in df.iterrows():
        shop_name = row['Shop Name']
        print(f"Extracting coordinates for: {shop_name}")
        
        latitude, longitude = get_latitude_longitude(driver, shop_name)
        latitudes.append(latitude)
        longitudes.append(longitude)
        
        time.sleep(2) 
    
    # Add the latitude and longitude to the DataFrame
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    
#     # Save the updated DataFrame back to a CSV file
    updated_csv_file_path = csv_file_path.replace('dallas_vape_shops.csv', 'dallas_vape_shops_with_lat_long.csv')
    
    df.to_csv("123news.csv", index=False)
    
    # Close the WebDriver
    driver.quit()
    
    return updated_csv_file_path

# Usage
csv_file_path = 'dallas_vape_shops.csv'  # Replace with your CSV file path

updated_csv_file_path = scrape_lat_long(csv_file_path)

# print(f"Updated CSV file saved to: {updated_csv_file_path}")

In [10]:
# lat_long=pd.read_csv('new.csv')

In [15]:
# lat_long.head(20)

In [7]:
# df3.drop(columns=['Latitude','Longitude'],inplace=True)

In [8]:
# directory = 'output scrapping'
# filename = 'Dallas_vape_data.csv'
# filepath = os.path.join(directory, filename)

# if not os.path.exists(directory):
#     os.makedirs(directory)

# df.to_csv(filepath, index=False)

In [9]:
# df4.to_csv('test_latlong')

WEBSITE URLs INSECTING

In [13]:
new_df=pd.read_csv('dallas_vape_shops_with_lat_long.csv')

In [14]:
for i in new_df['Website']:
    print(i)

nan
nan
nan
nan
nan
nan
karmatx.us
nan
nan
vapecity.us
smokeshopindallas.com
txhouseofvapes.com
the9thcloudsmokeshop.com
nan
nan
vapedz.com
nan
nan
nan
nan
nan
nan
vapedz.com
nan
nan
nan
nan
vapenvapor.com
worldofsmokenvape.com
nan
vapecavetx.com
nan
nan
nan
nan
worldofsmokenvape.com
worldofsmokenvape.com
vapecraftco.com
nan
nan
nan
artisanvaporcompany.com
nan
nan
nan
nan
nan
nan
vapegodshop.com
worldofsmokenvape.com
worldofsmokenvape.com
nan
vapeplus.info
nan
nan
nan
nan
nan
nan
nan
nan
smokeandvaporcbd.com
nan
nan
nan
klowdzvapor.com
nan
sublimsmokedfw.com
nan
nan
worldofsmokenvape.com
nan
nan
nan
worldofsmokenvape.com
nan
vapeplus.info
nan
nan
nan
nan
artisanvaporcompany.com
nan
uptownvapeandsmoke.com
nan
vapegodshop.com
artisanvaporcompany.com
vapecraftco.com
nan
artisanvaporcompany.com
nan
nan
nan
nan
nan
3dsmokeshop.net
vaporgalleriatarrantftworth.com
nan
artisanvaporcompany.com
nan
vapeshackusa.com
nan
nan
nan
nan
vapor-arena.com
krazysmoke.com
nan
nan
facebook.com
nan
nan
nan
n

In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time

# Define a regular expression pattern for email addresses
email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')

# List of domains to exclude (social media sites)
excluded_domains = {
    'facebook.com', 'twitter.com', 'instagram.com',
    'linkedin.com', 'youtube.com', 'tiktok.com'
}

# Function to check if email domain is in the excluded list
def is_excluded_email(email):
    domain = email.split('@')[-1]
    return any(excluded_domain in domain for excluded_domain in excluded_domains)

# Function to check if URL is a social media link
def is_social_media_url(url):
    return any(excluded_domain in url for excluded_domain in excluded_domains)

# Function to extract emails from a single URL
def extract_emails_from_url(url):
    service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    filtered_emails = []
    try:
        driver.get(url)
        time.sleep(5)
        page_text = driver.find_element(By.XPATH, '/html/body').text
        emails = email_pattern.findall(page_text)
        filtered_emails = [email for email in emails if not is_excluded_email(email)]
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
    finally:
        driver.quit()
    return url, filtered_emails

# Generator function to process URLs
def url_generator(df):
    for url in df['Website']:
        if pd.notnull(url) and isinstance(url, str) and url.strip() != "":
            yield url if url.startswith('http') else 'http://' + url

# Initialize a list to store the results
email_results = []

# Create a generator for URLs
url_gen = url_generator(new_df)

# Process URLs using the generator
for url in url_gen:
    if is_social_media_url(url):
        continue
    url, emails = extract_emails_from_url(url)
    email_results.append({'URL': url, 'Emails': ', '.join(emails)})

# Convert the results into a DataFrame
email_df = pd.DataFrame(email_results)

# Save the results to a new CSV file
email_df.to_csv('extracted_emails.csv', index=False)

# Print the DataFrame with URLs and filtered emails
print(email_df)


Error processing URL http://3dsmokeshop.net: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF67A4FEEB2+31554]
	(No symbol) [0x00007FF67A477EE9]
	(No symbol) [0x00007FF67A33872A]
	(No symbol) [0x00007FF67A330782]
	(No symbol) [0x00007FF67A321294]
	(No symbol) [0x00007FF67A322B02]
	(No symbol) [0x00007FF67A3215C1]
	(No symbol) [0x00007FF67A320DA1]
	(No symbol) [0x00007FF67A320CE0]
	(No symbol) [0x00007FF67A31EC6D]
	(No symbol) [0x00007FF67A31F2AC]
	(No symbol) [0x00007FF67A33B501]
	(No symbol) [0x00007FF67A3CD33E]
	(No symbol) [0x00007FF67A3AD02A]
	(No symbol) [0x00007FF67A3CC977]
	(No symbol) [0x00007FF67A3ACDD3]
	(No symbol) [0x00007FF67A37A33B]
	(No symbol) [0x00007FF67A37AED1]
	GetHandleVerifier [0x00007FF67A808B2D+3217341]
	GetHandleVerifier [0x00007FF67A855AF3+3532675]
	GetHandleVerifier [0x00007FF67A84B0F0+3489152]
	GetHandleVerifier [0x00007FF67A5AE786+750614]
	(No symbol) [0x00007FF67A48376F]
	(N

FINDING IMAGES 

In [52]:
# # correct images

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# import time

# def scrape_images(shop_name):
    
#     driver.get("https://www.google.com/maps")
    
#     try:
#         # Locate search box and enter the shop name
#         search_box = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
#         )
#         search_box.clear()
#         search_box.send_keys(shop_name)
#         search_box.send_keys(Keys.RETURN)
        
#         # Wait for the results to load
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]"))
#         )

#         # Click on the 'See all images' button if available
#         try:
#             see_all_images_button = WebDriverWait(driver, 5).until(
#                 EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button"))
#             )
#             see_all_images_button.click()
#         except:
#             pass
        
#         # Wait for images to load
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
#         )
        
#         # Collect image links
#         image_links = []
        
#         while len(image_links) < 20:
#             try:
#                 image_container = WebDriverWait(driver, 10).until(
#                     EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
#                 )
                
#                 images = image_container.find_elements(By.XPATH, ".//a/div[1]")
                
#                 for image in images:
#                     if len(image_links) >= 20:
#                         break
#                     img_src = image.get_attribute("style").split('("')[1].split('")')[0]
#                     if img_src not in image_links:
#                         image_links.append(img_src)
                
#                 # Scroll to load more images
#                 driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", image_container)
#                 time.sleep(2)  # Wait for more images to load

#             except Exception as e:
#                 print(f"Error during image collection: {e}")
#                 break
        
#         return image_links

#     except Exception as e:
#         print(f"Error scraping images for {shop_name}: {e}")
#         return []

# # Main script
# if __name__ == "__main__":
    
#     shop_name = "ktm vape city"
    
#     # Setup WebDriver
#     service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")
#     driver = webdriver.Chrome(service=service)
#     driver.maximize_window()
    
#     print(f"Scraping images for {shop_name}")
#     images = scrape_images(shop_name)
    
#     driver.quit()
    
#     # Print the collected images
#     print(f"\nImages for {shop_name}:")
#     for img in images:
#         print(img)


Scraping images for ktm vape city
Error during image collection: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6B588EEB2+31554]
	(No symbol) [0x00007FF6B5807EE9]
	(No symbol) [0x00007FF6B56C872A]
	(No symbol) [0x00007FF6B569D995]
	(No symbol) [0x00007FF6B57444D7]
	(No symbol) [0x00007FF6B575C051]
	(No symbol) [0x00007FF6B573CDD3]
	(No symbol) [0x00007FF6B570A33B]
	(No symbol) [0x00007FF6B570AED1]
	GetHandleVerifier [0x00007FF6B5B98B2D+3217341]
	GetHandleVerifier [0x00007FF6B5BE5AF3+3532675]
	GetHandleVerifier [0x00007FF6B5BDB0F0+3489152]
	GetHandleVerifier [0x00007FF6B593E786+750614]
	(No symbol) [0x00007FF6B581376F]
	(No symbol) [0x00007FF6B580EB24]
	(No symbol) [0x00007FF6B580ECB2]
	(No symbol) [0x00007FF6B57FE17F]
	BaseThreadInitThunk [0x00007FFE4D47257D+29]
	RtlUserThreadStart [0x00007FFE4F5EAF28+40]


Images for ktm vape city:
https://lh5.googleuserconten

In [54]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# import time
# import csv

# def scrape_images(shop_name):
#     driver.get("https://www.google.com/maps")
    
#     try:
#         # Locate search box and enter the shop name
#         search_box = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
#         )
#         search_box.clear()
#         search_box.send_keys(shop_name)
#         search_box.send_keys(Keys.RETURN)
        
#         # Wait for the results to load
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "//div[@role='main']"))
#         )

#         # Click on the 'See all images' button if available
#         try:
#             see_all_images_button = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button"))
#             )
#             see_all_images_button.click()
#         except Exception as e:
#             print(f"'See all images' button not found for {shop_name}: {e}")
#             return []
        
        

#         # Wait for images to load
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
#         )
        
#         # Collect image links
#         image_links = []
        
#         while len(image_links) < 10:
#             try:
#                 image_container = WebDriverWait(driver, 10).until(
#                     EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
#                 )
                
#                 images = image_container.find_elements(By.XPATH, ".//a/div[1]")
                
#                 for image in images:
#                     if len(image_links) >= 10:
#                         break
#                     img_src = image.get_attribute("style").split('("')[1].split('")')[0]
#                     if img_src and img_src not in image_links:
#                         image_links.append(img_src)
                
#                 # Scroll to load more images if needed
#                 driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", image_container)
#                 time.sleep(2)  # Wait for more images to load

#             except Exception as e:
#                 print(f"Error during image collection for {shop_name}: {e}")
#                 break
        
#         return image_links

#     except Exception as e:
#         print(f"Error scraping images for {shop_name}: {e}")
#         return []

# def clear_search_box():
#     try:
#         search_box = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
#         )
        
#         search_box.clear()
        
#     except Exception as e:
#         print(f"Error clearing search box: {e}")
        
# def save_to_csv(data, filename):
#     with open(filename, mode='w', newline='', encoding='utf-8') as file:
#         writer = csv.writer(file)
#         writer.writerow(["shop_name", "image_links"])
        
#         for shop_name, images in data.items():
            
#             writer.writerow([shop_name, ", ".join(images)])

# # Main script
# if __name__ == "__main__":
    
#     # List of shop names
#     shop_names = ["Love n Peace Smoke & vape shop", "The 9th Cloud Smoke & Vape","Vapor Lab - Supreme Smoke and Vape Little Elm","Wylie Vape"]
    
#     # Setup WebDriver
#     service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")
#     driver = webdriver.Chrome(service=service)
#     driver.maximize_window()
    
#     # Dictionary to hold the shop names and their corresponding image links
#     scraped_data = {}
    
#     for shop_name in shop_names:
#         print(f"Scraping images for {shop_name}")
#         images = scrape_images(shop_name)
#         scraped_data[shop_name] = images
        
#         # Clear the search and proceed to the next shop
#         try:
#             search_box = WebDriverWait(driver, 20).until(
#                 EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
#             )
#             search_box.clear()
#         except Exception as e:
#             print(f"Error clearing search box: {e}")
    
#     # Save the scraped data to a CSV file
#     save_to_csv(scraped_data, 'nepa_test_img.csv')
    
#     driver.quit()  
    
# #     done wlaa code

Scraping images for Love n Peace Smoke & vape shop
Error clearing search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B588EEB2+31554]
	(No symbol) [0x00007FF6B5807EE9]
	(No symbol) [0x00007FF6B56C872A]
	(No symbol) [0x00007FF6B5718434]
	(No symbol) [0x00007FF6B571853C]
	(No symbol) [0x00007FF6B575F6A7]
	(No symbol) [0x00007FF6B573D06F]
	(No symbol) [0x00007FF6B575C977]
	(No symbol) [0x00007FF6B573CDD3]
	(No symbol) [0x00007FF6B570A33B]
	(No symbol) [0x00007FF6B570AED1]
	GetHandleVerifier [0x00007FF6B5B98B2D+3217341]
	GetHandleVerifier [0x00007FF6B5BE5AF3+3532675]
	GetHandleVerifier [0x00007FF6B5BDB0F0+3489152]
	GetHandleVerifier [0x00007FF6B593E786+750614]
	(No symbol) [0x00007FF6B581376F]
	(No symbol) [0x00007FF6B580EB24]
	(No symbol) [0x00007FF6B580ECB2]
	(No symbol) [0x00007FF6B57FE17F]
	BaseThreadInitThunk [0x00007FFE4D47257D+29]
	RtlUserThreadStart [0x00007FFE4F5EAF28+40]

Scraping images for The 9th Cloud Smoke & Vape
Error during image collection for The 9th Cloud Sm

PUSH ONLY THIS CODE BY PUTTING INTO NEW.PY FILE

In [18]:
# final done wlaa code push th

import os
import csv
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def scrape_images(shop_name):
    driver.get("https://www.google.com/maps")
    
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
        )
        search_box.clear()
        search_box.send_keys(shop_name)
        search_box.send_keys(Keys.RETURN)
        
        # Wait for the results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='main']"))
        )

        image_links = []

        # Try to click on the 'See all images' button directly
        try:
            see_all_images_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button"))
            )
            see_all_images_button.click()
        
        except Exception as e:
            logging.warning(f"Direct 'See all images' button not found for {shop_name}: {e}")
            
            try:
                first_product = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[3]/div/a"))
                )
                first_product.click()

                # Wait for the details page and click the "See all images" button
                see_all_images_button_details = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[1]/div[1]/button"))
                )
                see_all_images_button_details.click()
                
            except Exception as e:
                logging.error(f"Error clicking 'See all images' button from details page: {e}")
                return []

        # Wait for images to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
        )
        
        while len(image_links) < 10:
            
            try:
                # Try scrolling down to load more images
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
                time.sleep(2)  # Wait for more images to load
                
                image_container = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
                )
                
                images = image_container.find_elements(By.XPATH, ".//a/div[1]")
                
                for image in images:
                    if len(image_links) >= 20:
                        break
                    img_src = image.get_attribute("style").split('("')[1].split('")')[0]
                    if img_src and img_src not in image_links:
                        image_links.append(img_src)
                    
                # Additional scrolling if needed
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", image_container)
                time.sleep(2)  # Wait for more images to load
                
                # If fewer than 20 images found and no more images loaded, stop
                if len(image_links) >= 20 or not images:
                    break
            
            except Exception as e:
                logging.error(f"Error during image collection for {shop_name}: {e}")
                break
            
        return image_links

    except Exception as e:
        logging.error(f"Error scraping images for {shop_name}: {e}")
        return []

def clear_search_box():
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
        )
        
        search_box.clear()
        search_box.send_keys(Keys.RETURN)  # Ensure the search is reset
        
        # Wait until the search box is cleared and ready for the next input
        WebDriverWait(driver, 5).until(
            EC.text_to_be_present_in_element_value((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"), "")
        )
        
    except Exception as e:
        logging.error(f"Error clearing search box: {e}")

def save_to_csv(data):
    with open("xxx20.csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["shop_name", "image_links"])
        for shop_name, images in data.items():
            writer.writerow([shop_name, ", ".join(images)])

# Main script
if __name__ == "__main__":
    
    # List of shop names
    
    shop_names = ["The 9th Cloud Smoke & Vape","Smoke & Vape DZ", "Vapor Lab - Supreme Smoke and Vape Little Elm", "Wylie Vape"]
    
    # Setup WebDriver
    service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")
    
    driver = webdriver.Chrome(service=service)
    
    driver.maximize_window()
    
    # Dictionary to hold the shop names and their corresponding image links
    scraped_data = {}
    
    for shop_name in shop_names:
        logging.info(f"Scraping images for {shop_name}")
        images = scrape_images(shop_name)
        scraped_data[shop_name] = images
        
        # Clear the search box and ensure it's ready for the next input
        clear_search_box()
        
        # Wait a bit to ensure the page is ready before starting the next search
        time.sleep(2)
    
    # Save the scraped data to a CSV file
    save_to_csv(scraped_data)
    
    driver.quit()


2024-07-24 23:37:18,107 - INFO - Scraping images for The 9th Cloud Smoke & Vape
2024-07-24 23:37:45,819 - ERROR - Error clearing search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6ED9FEEB2+31554]
	(No symbol) [0x00007FF6ED977EE9]
	(No symbol) [0x00007FF6ED83872A]
	(No symbol) [0x00007FF6ED888434]
	(No symbol) [0x00007FF6ED88853C]
	(No symbol) [0x00007FF6ED8CF6A7]
	(No symbol) [0x00007FF6ED8AD06F]
	(No symbol) [0x00007FF6ED8CC977]
	(No symbol) [0x00007FF6ED8ACDD3]
	(No symbol) [0x00007FF6ED87A33B]
	(No symbol) [0x00007FF6ED87AED1]
	GetHandleVerifier [0x00007FF6EDD08B2D+3217341]
	GetHandleVerifier [0x00007FF6EDD55AF3+3532675]
	GetHandleVerifier [0x00007FF6EDD4B0F0+3489152]
	GetHandleVerifier [0x00007FF6EDAAE786+750614]
	(No symbol) [0x00007FF6ED98376F]
	(No symbol) [0x00007FF6ED97EB24]
	(No symbol) [0x00007FF6ED97ECB2]
	(No symbol) [0x00007FF6ED96E17F]
	BaseThreadInitThunk [0x00007FFE4D47257D+29]
	RtlUserThreadStart [0x00007FFE4F5EAF28+40]

2024-07-24 23:37:47,829 - INFO - S

In [ ]:
# optimize code

import os
import csv
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def scrape_images(shop_name):
    driver.get("https://www.google.com/maps")
    
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
        )
        search_box.clear()
        search_box.send_keys(shop_name)
        search_box.send_keys(Keys.RETURN)
        
        # Wait for the results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='main']"))
        )

        image_links = []

        # Try to click on the 'See all images' button directly
        try:
            see_all_images_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button"))
            )
            see_all_images_button.click()
        
        except Exception as e:
            logging.warning(f"Direct 'See all images' button not found for {shop_name}: {e}")
            
            try:
                first_product = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[3]/div/a"))
                )
                first_product.click()

                # Wait for the details page and click the "See all images" button
                see_all_images_button_details = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[1]/div[1]/button"))
                )
                see_all_images_button_details.click()
                
            except Exception as e:
                logging.error(f"Error clicking 'See all images' button from details page: {e}")
                return []

        # Wait for images to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]"))
        )

        # Use list comprehension and generators to avoid loops
        image_links = [
            image.get_attribute("style").split('("')[1].split('")')[0]
            for _ in range(10)
            for image in WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, ".//a/div[1]"))
            )
            if image.get_attribute("style").split('("')[1].split('")')[0] not in image_links
        ][:10]

        return image_links

    except Exception as e:
        logging.error(f"Error scraping images for {shop_name}: {e}")
        return []

def clear_search_box():
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"))
        )
        
        search_box.clear()
        search_box.send_keys(Keys.RETURN)  # Ensure the search is reset
        
        # Wait until the search box is cleared and ready for the next input
        WebDriverWait(driver, 5).until(
            EC.text_to_be_present_in_element_value((By.XPATH, "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input"), "")
        )
        
    except Exception as e:
        logging.error(f"Error clearing search box: {e}")

def save_to_csv(data):
    with open("scraped_images.csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["shop_name", "image_links"])
        writer.writerows((shop_name, ", ".join(images)) for shop_name, images in data.items())

# Main script
if __name__ == "__main__":
    
    # List of shop names
    shop_names = ["The 9th Cloud Smoke & Vape", "Smoke & Vape DZ", "Vapor Lab - Supreme Smoke and Vape Little Elm", "Wylie Vape"]
    
    # Setup WebDriver
    service = Service(executable_path=r"C:\Users\apil xetri\OneDrive\Desktop\SCRAP_SELENIUM\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    
    # Dictionary to hold the shop names and their corresponding image links
    scraped_data = {shop_name: scrape_images(shop_name) for shop_name in shop_names}
    
    # Clear the search box after scraping each shop
    [clear_search_box() or time.sleep(2) for _ in shop_names]
    
    # Save the scraped data to a CSV file
    save_to_csv(scraped_data)
    
    driver.quit()
